In [9]:
#nuclio: ignore
import nuclio

In [ ]:
%%nuclio cmd -c
pip install opencv-contrib-python
pip install pandas
pip install v3io_frames

In [4]:
import nuclio_sdk
import json
import os
import v3io_frames as v3f
from requests import post
import base64
import numpy as np
import pandas as pd
import cv2
import random
import string
from datetime import datetime

In [ ]:
%%nuclio env
DATA_PATH = /User/demos/demos/faces/dataset/
V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}

In [90]:
is_partitioned = True #os.environ['IS_PARTITIONED']
http_addr = 'http://3.12.0.129:31893'
def generate_file_name(current_time, is_partitioned):
    filename_str = current_time + '.jpg'
    if is_partitioned == "true":
        filename_str = current_time[:-4] + "/" + filename_str
    return filename_str

def generate_image_path(filename, is_unknown):
    file_name = filename
    if is_unknown:
        pathTuple = (os.environ['DATA_PATH'] + 'label_pending', file_name) 
    else:
        pathTuple = (os.environ['DATA_PATH'] + 'images', file_name)   
    path = "/".join(pathTuple)
    return path

def jpg_str_to_frame(encoded):
    jpg_original = base64.b64decode(encoded)
    jpg_as_np = np.frombuffer(jpg_original, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img

def save_image(encoded_img, path):
    frame = jpg_str_to_frame(encoded_img)
    directory = '/'.join(path.split('/')[:-1])
    if not os.path.exists(directory):
        os.mkdir(directory)
    cv2.imwrite(path, frame)
    
def write_to_kv(client, face, path, camera, time):
    rnd_tag = ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))
    name = face['name']
    label = face['label']
    encoding = face['encoding']
    
    new_row = {}  
    new_row = {'c' + str(i).zfill(3): encoding[i] for i in range(128)}
    if name != 'unknown': 
        new_row['label'] = label
        new_row['fileName'] = name.replace(' ', '_') + '_' + rnd_tag
    else:
        new_row['label'] = -1
        new_row['fileName'] = 'unknown_' + rnd_tag
            
    new_row['imgUrl'] = path
    new_row['camera'] = camera
    new_row['time'] = datetime.strptime(time, '%Y%m%d%H%M%S')
    new_row_df = pd.DataFrame(new_row, index=[0])
    new_row_df = new_row_df.set_index('fileName')
    print(new_row['fileName'])
    client.write(backend='kv', table='iguazio/demos/demos/faces/artifacts/encodings', dfs=new_row_df) #, save_mode='createNewItemsOnly')   
    
def init_context(context):
    setattr(context.user_data, 'client', v3f.Client("framesd:8081", container="users"))
    
def handler(context, event):
        context.logger.info('extracting metadata')
        body = json.loads(event.body)
        time = body['time']
        camera = body['camera']
        encoded_img = body['content']
        
        content = {'img': encoded_img}

        context.logger.info('calling model server')
        resp = context.platform.call_function('recognize-faces', event)
        faces = json.loads(resp.body)
        #print(faces)
        #resp = post(http_addr, event.body)
        #faces = json.loads(resp.content)
        
        context.logger.info('going through discovered faces')
        for face in faces:
            is_unknown = face['name'] == 'unknown'
            file_name = generate_file_name(time, is_partitioned) 
            path = generate_image_path(file_name, is_unknown)
            
            context.logger.info('saving image to file system')
            save_image(encoded_img, path)
            
            context.logger.info('writing data to kv')
            write_to_kv(context.user_data.client, face, path, camera, time)
            
        return faces

In [ ]:
#nuclio: end-code

In [91]:
# converts the notebook code to deployable function with configurations
from mlrun import code_to_function, mount_v3io
fn = code_to_function('video-api-server', kind='nuclio')

# set the API/trigger, attach the home dir to the function
fn.with_http(workers=2).apply(mount_v3io())

# set environment variables
fn.set_env('DATA_PATH', '/User/demos/demos/faces/dataset/')
fn.set_env('V3IO_ACCESS_KEY', os.environ['V3IO_ACCESS_KEY'])
addr = fn.deploy(project='default')

[mlrun] 2020-03-01 13:41:46,328 deploy started
[nuclio] 2020-03-01 13:41:47,414 (info) Building processor image
[nuclio] 2020-03-01 13:41:52,458 (info) Build complete
[nuclio] 2020-03-01 13:41:56,500 (info) Function deploy complete
[nuclio] 2020-03-01 13:41:56,505 done updating video-api-server, function address: 3.12.0.129:32567


In [92]:
content = {"content": "...", "time": "20200227123029", "camera": "Avis-MacBook-Pro.local"}
content = json.dumps(content)

In [33]:
import nuclio_sdk
import logging
from requests import post

In [93]:
http_addr = 'http://3.12.0.129:32567'
resp = post(http_addr, content)

In [94]:
print(resp.content)

b'[{"coords": [249, 723, 402, 571], "name": "Avi Asulin", "label": 7, "confidence": 0.9964138455805363, "encoding": [-0.18035422265529633, 0.2196407914161682, 0.0788130909204483, -0.033359747380018234, -0.13244231045246124, 0.016204120591282845, -0.07550269365310669, -0.07482492923736572, 0.11943216621875763, -0.06061772257089615, 0.1868422031402588, -0.018235396593809128, -0.13835304975509644, 0.0063789403066039085, -0.07986482977867126, 0.1241222396492958, -0.1336575150489807, -0.13132993876934052, -0.16216576099395752, -0.17146795988082886, -0.04342985898256302, 0.07359129190444946, -0.07418132573366165, 0.023209402337670326, -0.13722065091133118, -0.25905588269233704, -0.020879361778497696, -0.05859178304672241, 0.08169004321098328, -0.04155909642577171, 0.03641408681869507, -0.1217154935002327, -0.19838377833366394, -0.07040998339653015, 0.04163716733455658, 0.09475130587816238, -0.09406167268753052, -0.005833599716424942, 0.17608946561813354, -0.05297325924038887, -0.127356767654